# Analysis of computer players playing against each other using different strategies and difficulties

Gathering data simulating all combinations of players (about 5.8 M) and storing them inside `benchmark.csv`
(This takes quite a bit of time and processing power!):

In [ ]:
!python3 benchmark.py

Installing and importing packages:

In [ ]:
!python3 -m pip install -r requirements.txt

In [1]:
import pandas as pd

Reading data from `benchmark.csv`:

In [2]:
data = pd.read_csv(
    "benchmark.csv", sep=";", header=None, names=[
    'X_strategy',
    'O_strategy',
    'X_difficulty',
    'O_difficulty',
    'winner'
])

print(data)

           X_strategy    O_strategy  X_difficulty  O_difficulty winner
0        [0, 0, 0, 0]  [0, 0, 0, 0]             0             0      X
1        [0, 0, 0, 0]  [0, 0, 0, 0]             0             1      X
2        [0, 0, 0, 0]  [0, 0, 0, 0]             0             2      O
3        [0, 0, 0, 0]  [0, 0, 0, 0]             0             3      O
4        [0, 0, 0, 0]  [0, 0, 0, 0]             1             0      X
...               ...           ...           ...           ...    ...
5806075  [8, 6, 4, 2]  [7, 5, 3, 1]             2             3      O
5806076  [8, 6, 4, 2]  [7, 5, 3, 1]             3             0      X
5806077  [8, 6, 4, 2]  [7, 5, 3, 1]             3             1      X
5806078  [8, 6, 4, 2]  [7, 5, 3, 1]             3             2      T
5806079  [8, 6, 4, 2]  [7, 5, 3, 1]             3             3      T

[5806080 rows x 5 columns]


Summarizing data:

In [3]:
summarized = data.groupby([
    "X_difficulty",
    "O_difficulty",
    "winner"
]).size().reset_index().rename(columns={0: 'count'})

o_won = summarized.loc[summarized['winner'] == 'O']
x_won = summarized.loc[summarized['winner'] == 'X']
ties = summarized.loc[summarized['winner'] == 'T']

print("X won:")
print(x_won)
print()
print("O won:")
print(o_won)
print()
print("Tie:")
print(ties)


X won:
    X_difficulty  O_difficulty winner   count
2              0             0      X  185472
5              0             1      X  131520
8              0             2      X   43136
13             1             0      X  190144
16             1             1      X  134976
19             1             2      X   47648
24             2             0      X  260608
27             2             1      X  177760
30             2             2      X   95488
34             3             0      X  329200
36             3             1      X  258000
38             3             2      X  117376

O won:
    X_difficulty  O_difficulty winner   count
0              0             0      O  122112
3              0             1      O  152064
6              0             2      O  235104
9              0             3      O  283136
11             1             0      O  113984
14             1             1      O  139200
17             1             2      O  217440
20             1   

Creating pivot tables:

In [4]:
x_won_pivot = x_won.pivot(index='X_difficulty', columns='O_difficulty', values='count')
o_won_pivot = o_won.pivot(index='X_difficulty', columns='O_difficulty', values='count')
ties_pivot = ties.pivot(index='X_difficulty', columns='O_difficulty', values='count')

x_won_pivot[3] = [0, 0, 0, 0]
o_won_pivot.loc[len(o_won_pivot)] = [0, 0, 0, 0]

print("X won:")
print(x_won_pivot)
print()
print("O won:")
print(o_won_pivot)
print()
print("Tie:")
print(ties_pivot)

X won:
O_difficulty       0       1       2  3
X_difficulty                           
0             185472  131520   43136  0
1             190144  134976   47648  0
2             260608  177760   95488  0
3             329200  258000  117376  0

O won:
O_difficulty       0       1       2       3
X_difficulty                                
0             122112  152064  235104  283136
1             113984  139200  217440  265344
2              45568   60448   67776  135552
3                  0       0       0       0

Tie:
O_difficulty      0       1       2       3
X_difficulty                               
0             55296   79296   84640   79744
1             58752   88704   97792   97536
2             56704  124672  199616  227328
3             33680  104880  245504  362880


Adding pivot tables together:

In [5]:
summed = x_won_pivot.add(o_won_pivot).add(ties_pivot)
print(summed)

O_difficulty       0       1       2       3
X_difficulty                                
0             362880  362880  362880  362880
1             362880  362880  362880  362880
2             362880  362880  362880  362880
3             362880  362880  362880  362880


Calculating ratios:

In [6]:
x_won_ratios = x_won_pivot.div(summed)
o_won_ratios = o_won_pivot.div(summed)
ties_ratios = ties_pivot.div(summed)

print("X won:")
print(x_won_ratios)
print()
print("O won:")
print(o_won_ratios)
print()
print("Ties:")
print(ties_ratios)

X won:
O_difficulty         0         1         2    3
X_difficulty                                   
0             0.511111  0.362434  0.118871  0.0
1             0.523986  0.371958  0.131305  0.0
2             0.718166  0.489859  0.263139  0.0
3             0.907187  0.710979  0.323457  0.0

O won:
O_difficulty         0         1         2         3
X_difficulty                                        
0             0.336508  0.419048  0.647884  0.780247
1             0.314109  0.383598  0.599206  0.731217
2             0.125573  0.166578  0.186772  0.373545
3             0.000000  0.000000  0.000000  0.000000

Ties:
O_difficulty         0         1         2         3
X_difficulty                                        
0             0.152381  0.218519  0.233245  0.219753
1             0.161905  0.244444  0.269489  0.268783
2             0.156261  0.343563  0.550088  0.626455
3             0.092813  0.289021  0.676543  1.000000
